# Tokenization

In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.3MB/s 
     |████████████████████████████████| 901kB 34.8MB/s 
     |████████████████████████████████| 3.3MB 37.7MB/s 


## Loading a Turkish Pre-trained Tokenizer

In [3]:
from transformers import AutoModel, AutoTokenizer
tokenizerTUR = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased",)
print(f"VOC size is: {tokenizerTUR.vocab_size}")
print(f"The model is {type(tokenizerTUR)}")


VOC size is: 32000
The model is <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


## Loading an English Pre-trained Tokenizer

In [4]:
from transformers import AutoModel, AutoTokenizer
tokenizerEN = AutoTokenizer.from_pretrained("bert-base-uncased")
print(f"VOC size is: {tokenizerEN.vocab_size}")
print(f"The model is {type(tokenizerEN)}")


VOC size is: 30522
The model is <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [5]:
word_en="telecommunications"
print(f"is in Turkish Model ? {word_en in tokenizerTUR.vocab}")
print(f"is in English Model ? {word_en in tokenizerEN.vocab}")

is in Turkish Model ? False
is in English Model ? True


In [6]:
tokens=tokenizerTUR.tokenize(word_en)
tokens

['tel', '##eco', '##mm', '##un', '##ica', '##tions']

But, The pieces are in the Turkish model

In [7]:
[t in tokenizerTUR.vocab for t in tokens]

[True, True, True, True, True, True]

In [8]:
tokens= tokenizerEN.tokenize(word_en)
tokens

['telecommunications']

In [9]:
long_word_tur="Muvaffakiyetsizleştiricileştiriveremeyebileceklerimizdenmişsinizcesine"

'''
It means that “As though you happen to have been from among those whom we will not be able to easily/quickly make a maker of unsuccessful ones” 
'''

In [10]:
print(tokenizerTUR.tokenize(long_word_tur))

['muvaffak', '##iyet', '##siz', '##les', '##tir', '##ici', '##les', '##tir', '##iver', '##emeye', '##bilecekleri', '##mi', '##z', '##den', '##mis', '##siniz', '##cesine']


## Understanding Tokenization Algorithms

### Train tokenizers from scratch

let's load Shakespeare plays from gutenberg project

In [11]:
import nltk 
from nltk.corpus import gutenberg 
nltk.download('gutenberg') 
nltk.download('punkt') 
plays=['shakespeare-macbeth.txt','shakespeare-hamlet.txt','shakespeare-caesar.txt']
shakespeare=[" ".join(s) for ply in plays for s in gutenberg.sents(ply)]

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# We prepare a template for the post-processing 
# Some initial settings

In [12]:
from tokenizers.processors import TemplateProcessing
special_tokens= ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
temp_proc= TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", special_tokens.index("[CLS]")),
        ("[SEP]", special_tokens.index("[SEP]")),
    ],
)

## Training BPE

In [13]:
from tokenizers import Tokenizer
from tokenizers.normalizers import (Sequence,Lowercase, NFD, StripAccents)
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.decoders import BPEDecoder

# Instantiate BPE (Byte-Pair Encoding)
tokenizer = Tokenizer(BPE())

# a unicode normalizer, lowercasing and , replacing accents in order  :
# * Sequence : It composes multiple PreTokenizer that will be run in the given order
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])

# Whitespace: Splits on word boundaries using the regular expression \w+|[^\w\s]+ 
tokenizer.pre_tokenizer = Whitespace() 
tokenizer.decoder = BPEDecoder()
tokenizer.post_processor=temp_proc

We are ready to train the model 

In [14]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=5000, special_tokens= special_tokens)
tokenizer.train_from_iterator(shakespeare, trainer=trainer)
print(f"Trained vocab size: {tokenizer.get_vocab_size()}" )

Trained vocab size: 5000


In [15]:
# take a sentence from macbeth

In [16]:
sen= "Is this a dagger which I see before me, the handle toward my hand?"
sen_enc=tokenizer.encode(sen)
print(f"Output: {format(sen_enc.tokens)}")

Output: ['[CLS]', 'is', 'this', 'a', 'dagger', 'which', 'i', 'see', 'before', 'me', ',', 'the', 'hand', 'le', 'toward', 'my', 'hand', '?', '[SEP]']


In [17]:
sen_enc2=tokenizer.encode("Macbeth and Hugging Face")

In [18]:
print(f"Output: {format(sen_enc2.tokens)}")

Output: ['[CLS]', 'macbeth', 'and', 'hu', 'gg', 'ing', 'face', '[SEP]']


In [19]:
# Let us pass  two sentences

In [20]:
two_enc=tokenizer.encode("I like Hugging Face!","He likes Macbeth!")

In [21]:
print(f"Output: {format(two_enc.tokens)}")

Output: ['[CLS]', 'i', 'like', 'hu', 'gg', 'ing', 'face', '!', '[SEP]', 'he', 'likes', 'macbeth', '!', '[SEP]']


In [22]:
tokenizer.model.save('.')

['./vocab.json', './merges.txt']

In [23]:
!wc -l ./merges.txt

4948 ./merges.txt


In [24]:
!head -6 ./merges.txt

#version: 0.2 - Trained by `huggingface/tokenizers`
t h
o u
a n
th e
r e


In [25]:
!head -1000 ./merges.txt| tail -5

ch ance
si g
your s
ti a
po int


In [26]:
# Save and Load Tokenizer

In [27]:
tokenizer.save("MyBPETokenizer.json")
tokenizerFromFile=Tokenizer.from_file("MyBPETokenizer.json")
sen_enc3 = tokenizerFromFile.encode("I like HuggingFace and Macbeth")
print(f"Output: {format(sen_enc3.tokens)}")

Output: ['[CLS]', 'i', 'like', 'hu', 'gg', 'ing', 'face', 'and', 'macbeth', '[SEP]']


## Training WordPiece

In [28]:
from tokenizers.models import WordPiece
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers.normalizers import BertNormalizer 

#BERT normalizer includes cleaning the text, handling accents, chinese chars and lowercasing

tokenizer = Tokenizer(WordPiece())
tokenizer.normalizer=BertNormalizer()
tokenizer.pre_tokenizer = Whitespace()

tokenizer.decoder= WordPieceDecoder()

In [29]:
from tokenizers.trainers import WordPieceTrainer
trainer = WordPieceTrainer(vocab_size=5000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

tokenizer.train_from_iterator(shakespeare, trainer=trainer)
output = tokenizer.encode(sen)
print(output.tokens)

['is', 'this', 'a', 'dagger', 'which', 'i', 'see', 'before', 'me', ',', 'the', 'hand', '##le', 'toward', 'my', 'hand', '?']


In [30]:
# let us use WordPiece Decoder to treat the sentences properly.

In [31]:
tokenizer.decode(output.ids)

'is this a dagger which i see before me, the handle toward my hand?'

In [32]:
# force the model to produce UNK tokens

In [33]:
tokenizer.encode("Kralsın aslansın Macbeth!").tokens

['[UNK]', '[UNK]', 'macbeth', '!']

# Pre-made tokenizers 
* CharBPETokenizer: The original BPE
* ByteLevelBPETokenizer: The byte level version of the BPE
* SentencePieceBPETokenizer: A BPE implementation compatible with the one used by SentencePiece
* BertWordPieceTokenizer: The famous Bert tokenizer, using WordPiece

In [34]:
# Fast Tokenizers optimized for Research and Production

In [35]:
from tokenizers import (ByteLevelBPETokenizer,
                            CharBPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)

In [36]:
tokenizer= SentencePieceBPETokenizer()
print(tokenizer.normalizer)
print(tokenizer.pre_tokenizer)
print(tokenizer.decoder)
print(tokenizer.post_processor)

None


In [37]:
tokenizer= BertWordPieceTokenizer()
print(tokenizer.normalizer)
print(tokenizer.pre_tokenizer)
print(tokenizer.decoder)
print(tokenizer.post_processor)

None
